In [3]:
import requests
from bs4 import BeautifulSoup

def request_douban(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
    except requests.RequestException:
        return None

C:\Users\kenshinpg\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### 先把所有的小组评论抓下来

In [4]:
def spider_links(page):
    url = 'https://www.douban.com/group/593625/discussion?start={}'.format(str(page*25))
    html = request_douban(url)
    soup = BeautifulSoup(html, 'lxml')
    list_ = soup.find(class_='olt').find_all('tr')
    page_link = []; page_title = []
    for item in list_:
        try:
            page_link.append(item.find('a').get('href'))
            page_title.append(item.find('a').get('title'))
        except:
            continue

### 再抓取页面内部的评论

In [201]:
def spider_page(url):
    # page_link = 'https://www.douban.com/group/topic/138987835/'
    html = request_douban(url)
    soup = BeautifulSoup(html, 'lxml')
    for item in soup.find(type="application/ld+json"):
        try:
            page_author_diag = json.loads(item)['text']
        except:
            continue
    list_ = soup.find_all(class_='clearfix comment-item reply-item')
    page_comments = []
    for item in list_:
        try:
            page_comments.append(item.find('p').contents[0])
        except:
            continue
    return page_author_diag, page_comments

In [284]:
import json

def json_load(file_path):
    with open(file_path) as json_file:
        return json.load(json_file)
    
output_json = json_load('./static/output.json')

In [285]:
import pandas as pd

output_json

{'https://www.douban.com/group/593625/': {'https://www.douban.com/group/topic/133073944/': {'author_diag': '',
   'comments': ['碎碎平安',
    '听说葡萄牙还是哪个国家过年必须摔旧碗，寓意除旧迎新，你摔掉的明明是今年的霉运！',
    '你真是摔到掉渣呀，今年桃花运一定很旺',
    '谢谢各位'],
   'title': '今天不小心摔了个碗～说是正月里摔碗不好～特意来这里说破它～'},
  'https://www.douban.com/group/topic/135268374/': {'author_diag': '但我很普通嗚嗚',
   'comments': ['这说明你的品味相当好啊 棒棒哒',
    '你一定也相当优秀，加油，去找到自己的幸福吧',
    '有眼光',
    '说明你自己本身也很优秀呀',
    '你是什么人你就会喜欢什么样的人',
    '眼光好，且有向优秀者靠拢的决心。很棒，你会越来越好，就算不能在一起，你会变得比原来好',
    '你一定可以的啦 打起精神来  预祝楼楼幸福美满',
    '这样你以后就会变得更优秀了',
    '我和你一样，他也很优秀，所以我更要努力了呀！你也是哦！',
    '努力变成能与他比肩的人吧',
    '说明你对优秀的向往',
    '赶紧追，过了这村没这店，男生都怕女生一直追',
    '说明你相当有眼光啊，没事大胆上，有条件上，没条件创造条件上，万一他其实喜欢你呢',
    '眼光不错',
    '眼光好啊！上啊'],
   'title': '喜歡上一位優秀的男生 求表揚'},
  'https://www.douban.com/group/topic/135593199/': {'author_diag': '',
   'comments': ['我先花了一天时间背了140个基础词根，记了词根以后再记单词就非常容易了',
    '你也太酷了   🙂   加油↖(^ω^)↗',
    '哇啊6666！太厉害了！加油！！！！',
    '厉害',
    '请告诉我怎么做到的，如此有恒心毅力',
    '我先花了一天时